In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
import os
os.chdir("/gdrive/My Drive/Stock Market Research/nifty/standardized_minute/")

In [ ]:
#saving last 75 days for testing
def regression_data_Xy(number_of_days_to_save_for_testing,lstm_data_path,root):
  import pickle
  with open(lstm_data_path,'rb') as fh:
    regressor_train_val=pickle.load(fh)
  x=int(len(pd.read_csv((root+'file_directory.csv')))-number_of_days_to_save_for_testing)
  y=int(regressor_train_val.shape[0]/len(pd.read_csv((root+'file_directory.csv'))))
  regressor_train_val=regressor_train_val[:int(x*y)]
  
  new_temp=pd.DataFrame(data=np.array(regressor_train_val['X'].values.tolist()).reshape(-1,np.array(regressor_train_val['X'][0]).shape[0]*np.array(regressor_train_val['X'][0]).shape[1]))
  new_temp['y']=regressor_train_val['y']
  import pickle
  with open(root+'regresion_X'+str(np.array(regressor_train_val['X'][0]).shape[0])+'_y'+str(np.array(regressor_train_val['y'][0]).shape[0])+'test'+str(number_of_days_to_save_for_testing)+'_data.pickle','wb') as fh:
    pickle.dump(new_temp,fh)
  
  return "done"

regression_data_Xy(number_of_days_to_save_for_testing=75,lstm_data_path='/gdrive/My Drive/Stock Market Research/nifty/standardized_minute/lstm_X20_y5_data.pickle',root='/gdrive/My Drive/Stock Market Research/nifty/standardized_minute/')

'done'

In [ ]:
import pickle
with open('/gdrive/My Drive/Stock Market Research/nifty/standardized_minute/regresion_X20_y5test75_data.pickle','rb') as fh:
  X_y=pickle.load(fh)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_y.drop('y',axis=1), np.array(np.array(X_y['y']).tolist()), test_size=0.2, random_state=42)

from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
dt_param_grid = {'max_depth': [2,3],
                 "min_samples_split": [8,10,12],"min_samples_leaf": [80,90,100,110]}
clf=GridSearchCV(dt,param_grid=dt_param_grid ,n_jobs=-1,verbose=700,cv=3,return_train_score=True)
clf.fit(X_train,y_train)
print(clf.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   39.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  4.

In [ ]:
with open('decsiontree_classifier.pickle','wb') as fh:
  pickle.dump(clf,fh)

In [ ]:
with open('output_scaler.pickle','rb') as fh:
  output_scaler=pickle.load(fh)
params=clf.best_params_
dt=DecisionTreeRegressor(**params)
model=dt.fit(X_train,y_train)

for i, j  in zip(model.predict(X_test),y_test):
  print("{} {}".format(output_scaler.inverse_transform(np.array(i).reshape(-1,1)).reshape(-1,),output_scaler.inverse_transform(np.array(j).reshape(-1,1)).reshape(-1,)))

Streaming output truncated to the last 5000 lines.
[10767.69988974 10767.69562541 10767.69825046 10767.69760789
 10767.68937861] [10654.  10654.8 10652.2 10653.3 10654.3]
[11464.60668169 11464.57626447 11464.53710538 11464.50353508
 11464.48164478] [11293.4  11292.7  11292.15 11291.85 11290.45]
[8155.8056508  8155.80849843 8155.80652093 8155.81846934 8155.82068221] [8268.75 8269.45 8268.   8270.95 8271.95]
[8666.5287797  8666.52948631 8666.52387813 8666.52071681 8666.50140538] [8768.5  8769.6  8770.95 8767.35 8772.05]
[7611.18178785 7611.17365572 7611.17494845 7611.1860567  7611.18631036] [7458.15 7457.8  7460.75 7462.1  7461.35]
[8666.5287797  8666.52948631 8666.52387813 8666.52071681 8666.50140538] [8805.3  8806.05 8811.75 8809.85 8811.8 ]
[10767.69988974 10767.69562541 10767.69825046 10767.69760789
 10767.68937861] [10948.85 10951.1  10937.7  10937.4  10939.7 ]
[7611.18178785 7611.17365572 7611.17494845 7611.1860567  7611.18631036] [7386.95 7389.45 7392.8  7390.8  7392.6 ]
[7611.181

In [ ]:
clf.best_score_

0.05960452067139876

In [ ]:
clf.cv_results_

{'mean_fit_time': array([35.13753406, 33.99842167, 34.8023928 , 34.6968379 , 34.78785467,
        34.91821718, 35.61001102, 35.2691768 , 34.97823564, 35.14839387,
        35.12745643, 34.93500439, 50.53880151, 50.71001379, 50.51844478,
        50.68161956, 49.98285882, 50.25159009, 49.65476418, 50.23131331,
        50.70169338, 51.154128  , 50.40877088, 49.0253094 ]),
 'mean_score_time': array([0.14768712, 0.13839451, 0.13626981, 0.14083982, 0.13619741,
        0.14006313, 0.13484804, 0.14449843, 0.13590837, 0.13588293,
        0.13642812, 0.13745777, 0.14333836, 0.14103206, 0.14112401,
        0.14019267, 0.13935145, 0.13560796, 0.14111884, 0.13648979,
        0.14122415, 0.14063541, 0.13825742, 0.12254389]),
 'mean_test_score': array([0.05960452, 0.05960452, 0.05960452, 0.05960452, 0.05960452,
        0.05960452, 0.05960452, 0.05960452, 0.05960452, 0.05960452,
        0.05960452, 0.05960452, 0.01621272, 0.01621272, 0.01621272,
        0.01621272, 0.01621272, 0.01621272, 0.01621272, 0

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(365040, 320)
(365040, 5)
(91260, 320)
(91260, 5)


In [ ]:
import tensorflow as tf
from tensorflow import keras
target=X_y['y']
dataset = tf.data.Dataset.from_tensor_slices(np.array(X_y.values), np.array((target.values)))
train_dataset = dataset.shuffle(len(df)).batch(1)


TypeError: ignored

In [ ]:
def build_model():
  model = keras.Sequential([
      keras.layers.Input(shape=(None,320)),
      keras.layers.Dense(500, activation='relu'),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(250, activation='relu'),
      keras.layers.Dropout(0.1),
      keras.layers.Dense(5)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, None, 500)         160500    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 500)         0         
_________________________________________________________________
dense_4 (Dense)              (None, None, 250)         125250    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 250)         0         
_________________________________________________________________
dense_5 (Dense)              (None, None, 5)           1255      
Total params: 287,005
Trainable params: 287,005
Non-trainable params: 0
_________________________________________________________________


In [ ]:
EPOCHS = 1000

history = model.fit(
  X_y.drop('y',axis=1), X_y['y'],
  epochs=EPOCHS, validation_split = 0.2, verbose=1)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

ValueError: ignored